In [11]:
%pip install chromadb pymupdf pillow gemma 
#pip install ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [12]:
%pip install ollama

Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [ ]:
#PDF Conversion to Handle Multiple PDFs
import os

def convert_pdfs_in_folder(folder_path, output_dir):
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
    
    all_image_paths = []
    
    for pdf_file in pdf_files:
        pdf_path = os.path.join(folder_path, pdf_file)
        # Convert each PDF to images
        image_paths = convert_pdf_to_images(pdf_path, output_dir)
        all_image_paths.extend(image_paths)
    
    return all_image_paths


In [3]:
#Convert PDF Pages to Images

import fitz  # PyMuPDF
from PIL import Image
import os

def convert_pdf_to_images(pdf_path, output_dir):
    # Open the provided PDF
    pdf_document = fitz.open(pdf_path)
    
    # Iterate over each page and convert to image
    image_paths = []
    for page_num in range(len(pdf_document)):
        # Get the page
        page = pdf_document.load_page(page_num)
        
        # Convert page to pixmap (image)
        pix = page.get_pixmap()
        
        # Convert to PIL Image for easier manipulation
        img = Image.frombytes("RGB", [pix.width, pix.height], pix.samples)
        
        # Create an image file path
        img_path = os.path.join(output_dir, f"page_{page_num+1}.png")
        img.save(img_path)
        image_paths.append(img_path)
    
    return image_paths


In [ ]:
#Modify OCR to Handle Multiple PDFs
def extract_text_from_all_pdfs(folder_path, output_dir):
    # Step 1: Convert all PDFs to images
    image_paths = convert_pdfs_in_folder(folder_path, output_dir)
    
    # Step 2: Extract text using OCR (Gemma)
    extracted_texts = extract_text_from_images(image_paths)
    
    return extracted_texts


In [4]:
#Apply OCR using Gemma
# from gemma import Gemma

# def extract_text_from_images(image_paths):
#     # Initialize Gemma OCR model
#     gemma_model = Gemma()

#     # Extract text from each image
#     extracted_texts = []
#     for img_path in image_paths:
#         text = gemma_model.extract_text(img_path)
#         extracted_texts.append(text)
    
#     return extracted_texts



#from ollama import Ollama
from PIL import Image
import base64
import io
import ollama

def image_to_base64(img_path):
    """Convert image to base64 string"""
    with open(img_path, "rb") as f:
        encoded = base64.b64encode(f.read()).decode("utf-8")
    return encoded

def extract_text_from_images_with_gemma(image_paths):
    """
    Use Gemma 3:12B via Ollama to extract text from images.
    """
    extracted_texts = []

    for img_path in image_paths:
        img_b64 = image_to_base64(img_path)
        
        # Prepare prompt for OCR
        prompt = f"Extract all text from this image and return as plain text:\n\n[image_data_base64:{img_b64}]"
        
        # Call Ollama Gemma model 
        response = ollama.chat(model="gemma3:12b", messages=[{"role":"user", "content": prompt}])
        
        extracted_text = response["message"]["content"]
        extracted_texts.append(extracted_text)
    
    return extracted_texts


In [8]:
#Save Extracted Text to .txt Files
# def save_extracted_text_as_txt(extracted_texts, output_dir):
#     """Save the extracted text from images into .txt files in a new folder."""
#     text_output_dir = os.path.join(output_dir, "extracted_texts")
#     if not os.path.exists(text_output_dir):
#         os.makedirs(text_output_dir)
    
#     for i, text in enumerate(extracted_texts):
#         file_path = os.path.join(text_output_dir, f"page_{i+1}.txt")
#         with open(file_path, "w", encoding="utf-8") as f:
#             f.write(text)
#     print(f"Extracted texts have been saved to {text_output_dir}")

# Function to save the extracted text as txt files
from IPython.display import display, Markdown

# Function to save the extracted text as txt files and display it
def save_extracted_text_as_txt(extracted_texts, output_dir):
    """Save the extracted text from images into .txt files in a new folder and display the text."""
    
    # Create directory for storing text files
    text_output_dir = os.path.join(output_dir, "extracted_texts")
    if not os.path.exists(text_output_dir):
        os.makedirs(text_output_dir)
    
    # Loop through each extracted text, save to file, and display it
    for i, text in enumerate(extracted_texts):
        file_path = os.path.join(text_output_dir, f"page_{i+1}.txt")
        
        # Save the text to the file
        with open(file_path, "w", encoding="utf-8") as f:
            f.write(text)
        
        # Display the text in the notebook output immediately after saving
        display(Markdown(f"### Extracted Text from Page {i+1}:"))
        display(Markdown(f"```\n{text}\n```"))

    print(f"Extracted texts have been saved to {text_output_dir}")



In [ ]:
#ChromaDB Storage to Include Multiple PDFs
def add_texts_to_chromadb_for_multiple_pdfs(extracted_texts, folder_path):
    # Get the ChromaDB collection
    collection = client.get_collection(name="pdf_knowledgebase")
    
    pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
    all_metadata = []
    
    # Step 1: Add each extracted text from multiple PDFs to ChromaDB
    for pdf_file in pdf_files:
        pdf_path = os.path.join(folder_path, pdf_file)
        extracted_text = extracted_texts.get(pdf_file, [])
        
        for page_num, text in enumerate(extracted_text):
            # Create metadata for each page
            metadata = {'pdf_name': pdf_file, 'page': page_num + 1}
            collection.add(
                documents=[text],  # Add the text of the page
                metadatas=[metadata],  # Add metadata
                ids=[f"{pdf_file}_page_{page_num + 1}"],  # Unique ID for each page
            )


In [6]:
%pip install --upgrade chromadb chroma-migrate


Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 23.0.1 -> 25.2
[notice] To update, run: python.exe -m pip install --upgrade pip


In [17]:
import chromadb
from chromadb.config import Settings

# Initialize a local Chroma client (database stored in ./chroma_db)
# client = chromadb.Client(
#     persist_directory="./chroma_db",  # folder where data is stored
#     chroma_db_impl="duckdb+parquet"   # local database backend
# )


#import chromadb
chroma_client = chromadb.Client()

# switch `create_collection` to `get_or_create_collection` to avoid creating a new collection every time
collection = chroma_client.get_or_create_collection(name="my_collection")

# switch `add` to `upsert` to avoid adding the same documents every time
collection.upsert(
    documents=[
        "This is a document about pineapple",
        "This is a document about oranges"
    ],
    ids=["id1", "id2"]
)

results = collection.query(
    query_texts=["This is a query document about florida"], # Chroma will embed this for you
    n_results=2 # how many results to return
)

print(results)

# # Create a collection (like a table)
# try:
#     collection = chroma_client.get_or_create_collection(name="my_collection")
#     #collection = client.get_collection("pdf_knowledgebase")
# except ValueError:
#     collection = client.create_collection("pdf_knowledgebase")

# # Add documents
# texts = ["This is page 1 text", "This is page 2 text"]
# metadata = [{"pdf_name": "example.pdf", "page": 1}, {"pdf_name": "example.pdf", "page": 2}]
# ids = ["example_1", "example_2"]

# collection.add(documents=texts, metadatas=metadata, ids=ids)

# # Query: Find similar text
# results = collection.query(
#     query_texts=["page 1"],
#     n_results=1
# )
# print(results)


C:\Users\Busra\.cache\chroma\onnx_models\all-MiniLM-L6-v2\onnx.tar.gz: 100%|██████████| 79.3M/79.3M [17:24<00:00, 79.7kiB/s]


{'ids': [['id2', 'id1']], 'embeddings': None, 'documents': [['This is a document about oranges', 'This is a document about pineapple']], 'uris': None, 'included': ['metadatas', 'documents', 'distances'], 'data': None, 'metadatas': [[None, None]], 'distances': [[1.1462137699127197, 1.3015384674072266]]}


In [ ]:
##ChromaDB Storage to Include pdfs

import chromadb

# Initialize Chroma client
chroma_client = chromadb.Client()

# Get or create a collection (avoiding creating a new one every time)
collection = chroma_client.get_or_create_collection(name="pdf_knowledgebase")

def add_text_to_chromadb(extracted_texts, metadata_list):
    """
    Add or update extracted text documents in ChromaDB.
    """
    # Prepare data for upserting
    ids = [f"{meta['pdf_name']}_page_{meta['page']}" for meta in metadata_list]
    
    # Upsert documents and their metadata
    collection.upsert(
        documents=extracted_texts,  # list of extracted text
        ids=ids,                    # unique IDs based on PDF name and page number
        metadatas=metadata_list     # metadata for each document
    )



In [ ]:
#Setting up the Chatbot (GPT-OSS 20B)
# from ollama import Ollama

# def gpt_oss_20b_query(query, collection):
#     # Retrieve the most relevant documents from ChromaDB
#     results = collection.query(query, n_results=5)  # fetch top 5 results

#     # Combine the retrieved documents' texts
#     context = " ".join([result['document'] for result in results['documents']])

#     # Send the query to the GPT-OSS model with context
#     model = Ollama(model="gpt-oss:20b")
#     response = model.chat(context + "\n" + query)
    
#     return response['text']



# import ollama

# def gpt_oss_20b_query(query, collection):
#     # Retrieve the most relevant documents from ChromaDB
#     results = collection.query(query, n_results=5)  # fetch top 5 results

#     # Combine the retrieved documents' texts
#     context = " ".join([result['document'] for result in results['documents']])

#     # Prepare the prompt for the model: Combining context and user query
#     prompt = context + "\n\nUser Query: " + query

#     # Call the local GPT-OSS 20B model via Ollama
#     response = ollama.chat(model="gpt-oss:20b", messages=[{"role": "user", "content": prompt}])

#     # Return the model's response
#     return response["message"]["content"]
#----------------------------------------------------------------------------------------

# import ollama

# def gpt_oss_20b_query(query, collection):
#     # Retrieve the most relevant documents from ChromaDB
#     results = collection.query(query, n_results=5)  # fetch top 5 results

#     # Correct extraction of documents from the results
#     # The results['documents'] is a list of lists, so extract the first list
#     documents = results['documents'][0]  # The first element contains the documents

#     # Combine the retrieved documents' texts into context
#     context = " ".join(documents)

#     # Prepare the prompt for the model: Combining context and user query
#     prompt = context + "\n\nUser Query: " + query

#     # Call the local GPT-OSS 20B model via Ollama
#     response = ollama.chat(model="gpt-oss:20b", messages=[{"role": "user", "content": prompt}])

#     # Return the model's response
#     return response["message"]["content"]

import ollama

def gpt_oss_20b_query(query, collection):
    # Retrieve the most relevant documents from ChromaDB based on the user's query
    results = collection.query(
        query_texts=[query],  # Pass query text directly
        n_results=5  # Fetch top 5 results
    )

    # Correct extraction of documents from the query results
    # The results['documents'] is a list of lists, so extract the first list
    documents = results['documents'][0]  # The first element contains the documents

    # Combine the retrieved documents' texts into context
    context = " ".join(documents)

    # Prepare the prompt for the model: Combining context and user query
    prompt = context + "\n\nUser Query: " + query

    # Call the local GPT-OSS 20B model via Ollama with the combined context and user query
    response = ollama.chat(model="gpt-oss:20b", messages=[{"role": "user", "content": prompt}])

    # Return the model's response
    return response["message"]["content"]


In [38]:
#Handle Deletion of PDFs and Associated Data
def remove_pdf_from_chromadb(pdf_name):
    # Get the collection
    collection = client.get_collection(name="pdf_knowledgebase")
    
    # Query to find all pages related to the PDF
    results = collection.query(f"{pdf_name}", n_results=100)
    
    # Remove the documents that belong to this PDF
    for result in results['documents']:
        collection.delete(ids=[result['id']])


In [ ]:
##Handle Deletion of PDFs and Associated Data with chromadb
def remove_pdf_from_chromadb(pdf_name):
    """Remove all pages related to the specified PDF from ChromaDB."""
    # Get the collection
    chroma_client = chromadb.Client()  # Create a client if not already done
    collection = chroma_client.get_or_create_collection(name="pdf_knowledgebase")
    
    # Query to find all pages related to the PDF (based on the pdf_name in metadata)
    results = collection.query(query_texts=[pdf_name], n_results=100)
    
    # Remove the documents that belong to this PDF
    for result in results['documents']:
        # Delete each document by its unique ID
        collection.delete(ids=[result['id']])
    
    print(f"All documents related to {pdf_name} have been removed from the collection.")


In [9]:
#testing the extracted texts


# Path for PDF and Output directory
pdf_path = "./PDF/return 2 pages.pdf"  
output_dir = "output"   

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Step 1: Convert PDF to Images
image_paths = convert_pdf_to_images(pdf_path, output_dir)

# Step 2: Apply OCR to Extract Text
extracted_texts = extract_text_from_images_with_gemma(image_paths)

In [10]:
# Step 3: Store Text in ChromaDB
# metadata = [{'pdf_name': pdf_path, 'page': i+1} for i in range(len(extracted_texts))]
# add_text_to_chromadb(extracted_texts, metadata) 

# Step 4: Save extracted text to .txt files

output_dir = "text_putput"  # Directory to save the text files

save_extracted_text_as_txt(extracted_texts, output_dir)


### Extracted Text from Page 1:

```
This is a base64 encoded image. Decoding it reveals an image of a stylized, colorful, and possibly abstract landscape. Here's a breakdown of what it likely depicts and a summary:

**Observations:**

*   **Landforms:** It seems to feature rolling hills, a suggestion of a river or lake, and possibly mountains or rock formations in the background. The landforms are highly stylized, not realistic.
*   **Colors:** A broad palette is used, mixing blues, greens, yellows, oranges, and purples. This suggests a dreamlike or fantastical atmosphere.
*   **Style:** The artwork has a graphic, almost digital look. It could be a vector illustration or have been digitally painted. The shapes are simplified and often blend into one another.
*   **Possible Theme:** The overall feeling is one of tranquility and natural beauty, but the abstract nature invites personal interpretation. It could be a scene from a fictional world or a symbolic representation of nature.



**Summary:**
The base64 string represents a colorful, abstract landscape image, likely intended to evoke a sense of peace and wonder.
```

### Extracted Text from Page 2:

```
This appears to be a binary blob, likely an image encoded as Base64.  Decoding it will reveal its true form, but without decoding it, it's just a string of characters.

Here's the likely interpretation:

*   **It's a Base64 Encoded Image:** The string is structured in a way that's characteristic of Base64 encoding. This means it represents image data that has been converted into a text string for safe transmission or storage.
*   **Decoding Required:** To see what the image *is*, you need to decode it. Online Base64 decoders (easily found with a search) can do this.  Common image formats like PNG or JPG are likely.

**To see the image, I need to decode it.** I cannot directly perform actions like decoding images in this text-based environment. Use a Base64 decoder to view the actual image.
```

Extracted texts have been saved to text_putput\extracted_texts


In [40]:
# pdf_path = "./PDF"
# output_dir = "output"

# # Step 1: Convert PDF to Images
# image_paths = convert_pdf_to_images(pdf_path, output_dir)

# # Step 2: Apply OCR to Extract Text
# extracted_texts = extract_text_from_images_with_gemma(image_paths)

# # Step 3: Store Text in ChromaDB
# metadata = [{'pdf_name': pdf_path, 'page': i+1} for i in range(len(extracted_texts))]
# add_text_to_chromadb(extracted_texts, metadata)

# # Step 4: Use Chatbot for Querying
# user_query = "What is the service about?"
# response = gpt_oss_20b_query(user_query, client.get_collection(name="pdf_knowledgebase"))
# print(response)

#------------------------------
# Example Usage
# pdf_path = "./PDF/Return Policy.pdf"  # Path to the folder with PDFs
# output_dir = "output"  # Directory where images will be saved

# # Step 1: Convert PDF to Images
# image_paths = convert_pdf_to_images(pdf_path, output_dir)

# # Step 2: Apply OCR to Extract Text
# extracted_texts = extract_text_from_images_with_gemma(image_paths)

# # Step 3: Store Text in ChromaDB
# metadata = [{'pdf_name': pdf_path, 'page': i+1} for i in range(len(extracted_texts))]
# add_text_to_chromadb(extracted_texts, metadata)

# # Step 4: Use Chatbot for Querying
# user_query = "What is the service about?"
# chroma_client = chromadb.Client()
# collection = chroma_client.get_or_create_collection(name="pdf_knowledgebase")
# response = gpt_oss_20b_query(user_query, collection)
# print(response)




#-----------------------------
# import os

# # Example Usage
# pdf_path = "./PDF/Return Policy.pdf"  # Path to the PDF
# output_dir = "output"  # Directory where images will be saved

# # Ensure the output directory exists
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# # Step 1: Convert PDF to Images
# image_paths = convert_pdf_to_images(pdf_path, output_dir)

# # Step 2: Apply OCR to Extract Text
# extracted_texts = extract_text_from_images_with_gemma(image_paths)

# # Step 3: Store Text in ChromaDB
# metadata = [{'pdf_name': pdf_path, 'page': i+1} for i in range(len(extracted_texts))]
# add_text_to_chromadb(extracted_texts, metadata)

# # Step 4: Use Chatbot for Querying
# user_query = "What is the service about?"
# chroma_client = chromadb.Client()
# collection = chroma_client.get_or_create_collection(name="pdf_knowledgebase")
# response = gpt_oss_20b_query(user_query, collection)
# print(response)

#------------------------------------------------
#------------------------------------------------
# import chromadb
# import numpy as np

# # Example Usage 
# pdf_path = "./PDF/return 2 pages.pdf"  # Path to the PDF
# output_dir = "output"  # Directory where images will be saved

# # Ensure the output directory exists
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# # Step 1: Convert PDF to Images
# image_paths = convert_pdf_to_images(pdf_path, output_dir)

# # Step 2: Apply OCR to Extract Text
# extracted_texts = extract_text_from_images_with_gemma(image_paths)

# # Step 3: Store Text in ChromaDB
# metadata = [{'pdf_name': pdf_path, 'page': i+1} for i in range(len(extracted_texts))]
# add_text_to_chromadb(extracted_texts, metadata)

# # Step 4: Use Chatbot for Querying

# # Query Text (no need to embed manually if Chroma handles it internally)
# user_query = "What is the service about?"

# # Chroma Client
# chroma_client = chromadb.Client()
# collection = chroma_client.get_or_create_collection(name="pdf_knowledgebase")

# # Get embeddings of the user query using Chroma's internal embedding model
# #query_embeddings = collection.get_embeddings([user_query])


# # Perform the query with ChromaDB (querying the text directly, Chroma will embed it internally)
# results = collection.query(
#     query_texts=[user_query],  # Pass query text directly
#     n_results=5  # Fetch top 5 results
# )

# # Perform the query with embeddings
# # results = collection.query(
# #     query_embeddings=query_embeddings,  # Use query embeddings here
# #     n_results=5  # Fetch top 5 results
# # )

# # Combine the retrieved documents' texts into context
# context = " ".join([result['document'] for result in results['documents']])

# # Generate the final response with GPT-OSS 20B model using the context
# response = gpt_oss_20b_query(user_query, collection)
# print(response)
#------------------------------------------------
#------------------------------------------------
#------------------------------------------------
# import chromadb
# import os

# # Example Usage
# pdf_path = "./PDF/return 2 pages.pdf"  # Path to the PDF
# output_dir = "output"  # Directory where images will be saved

# # Ensure the output directory exists
# if not os.path.exists(output_dir):
#     os.makedirs(output_dir)

# # Step 1: Convert PDF to Images (Assuming convert_pdf_to_images function is defined)
# image_paths = convert_pdf_to_images(pdf_path, output_dir)

# # Step 2: Apply OCR to Extract Text (Assuming extract_text_from_images_with_gemma function is defined)
# extracted_texts = extract_text_from_images_with_gemma(image_paths)

# # Step 3: Store Text in ChromaDB
# metadata = [{'pdf_name': pdf_path, 'page': i+1} for i in range(len(extracted_texts))]
# add_text_to_chromadb(extracted_texts, metadata)

# # Step 4: Use Chatbot for Querying

# # Query Text (no need to embed manually if Chroma handles it internally)
# user_query = "What is the service about?"

# # Chroma Client
# chroma_client = chromadb.Client()
# collection = chroma_client.get_or_create_collection(name="pdf_knowledgebase")

# # Get embeddings of the user query using Chroma's internal embedding model
# results = collection.query(
#     query_texts=[user_query],  # Pass query text directly
#     n_results=5  # Fetch top 5 results
# )

# # Correct the extraction of the documents from the query results
# # The results['documents'] is a list of lists, so we need to extract the first element
# documents = results['documents'][0]  # The first element contains the documents

# # Join the retrieved documents' texts into context
# context = " ".join(documents)

# # Now you can use the context to generate the response
# response = gpt_oss_20b_query(user_query, context)  # Assuming gpt_oss_20b_query is defined elsewhere

# print(response)

#------------------------------------------------
#------------------------------------------------
#------------------------------------------------
#------------------------------------------------
#------------------------------------------------

import chromadb
import os

# Example Usage
pdf_path = "./PDF/return 2 pages.pdf"  # Path to the PDF
output_dir = "output"  # Directory where images will be saved

# Ensure the output directory exists
if not os.path.exists(output_dir):
    os.makedirs(output_dir)

# Step 1: Convert PDF to Images (Assuming convert_pdf_to_images function is defined)
image_paths = convert_pdf_to_images(pdf_path, output_dir)

# Step 2: Apply OCR to Extract Text (Assuming extract_text_from_images_with_gemma function is defined)
extracted_texts = extract_text_from_images_with_gemma(image_paths)

# Step 3: Store Text in ChromaDB
metadata = [{'pdf_name': pdf_path, 'page': i+1} for i in range(len(extracted_texts))]
add_text_to_chromadb(extracted_texts, metadata)

# Step 4: Use Chatbot for Querying

# Query Text (no need to embed manually if Chroma handles it internally)
user_query = "What is the service about?"

# Chroma Client
chroma_client = chromadb.Client()
collection = chroma_client.get_or_create_collection(name="pdf_knowledgebase")

# Get embeddings of the user query using Chroma's internal embedding model
results = collection.query(
    query_texts=[user_query],  # Pass query text directly
    n_results=5  # Fetch top 5 results
)

# Correct the extraction of the documents from the query results
# The results['documents'] is a list of lists, so we need to extract the first element
documents = results['documents'][0]  # The first element contains the documents

# Join the retrieved documents' texts into context
context = " ".join(documents)

# Now you can use the context to generate the response
response = gpt_oss_20b_query(user_query, collection)  # Pass the collection instead of context

print(response)


ResponseError: model "gpt-oss-20b" not found, try pulling it first (status code: 404)

In [ ]:
#all
folder_path = "./PDF"  # Folder containing PDFs
output_dir = "output_images"  # Folder to store images

# Step 1: Extract text from all PDFs
extracted_texts = extract_text_from_all_pdfs(folder_path, output_dir)

# Create a dictionary to organize extracted text by PDF
organized_texts = {}
pdf_files = [f for f in os.listdir(folder_path) if f.endswith('.pdf')]
for pdf_file in pdf_files:
    extracted_text = extracted_texts  # Extract text for this PDF (adjust as needed)
    organized_texts[pdf_file] = extracted_text

# Step 2: Add the extracted texts to ChromaDB
add_texts_to_chromadb_for_multiple_pdfs(organized_texts, folder_path)

# Step 3: Querying the chatbot
user_query = "What is the service about?"
response = gpt_oss_20b_query(user_query, client.get_collection(name="pdf_knowledgebase"))
print(response)
